# Instructions

For Part A, you need to scrape IMDB web page to find out top movies sorted by user votes. For each movie, you need to pull :
- movie_id
- rank
- title 
- year
- rating
- runtime
- votes
- genres

The URL of an page that include movies released between 2018 and 2020 sorted by number of votes is: 

https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020


Please click the URL and investigate how you can pull movie_id, rank, title,... from the webpage. This webpage, however, only includes 50 movies. Hence, if you want to extract top 250 movies between 2018 and 2020 according to  the number of votes, you need to click “next” 4 times and parse 4 more pages. Fortunately, we can do this by modifying the URL a little. For example, the URL

https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=51&title_type=feature&year=2018,2020
(Note that start = 1 in the first URL, now start = 51) allows us to move on to the next page. Obviously, https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=101&title_type=feature&year=2018,2020
will lead us to the third page. 


**You need to write code after where I have <span style="color:red">'''  Your code here ...    '''.</span>**

***
Now let’s look at each function in detail. The parameter “top_number” in the function read_m_by_voting(first_year, last_year, top_number)  represents the top number of movies you want to retrieve. For example, read_m_by_voting(2018,2020,500) means that we want to extract top 500 movies released between 2018 and 2020, sorted by users' votes.

This function returns a list of dictionaries. Each dictionary represents one of the top movies, which could look like the following:

{   
   
      'movie_id': 'tt6324278',
      'title': 'Abominable',
      'year': '(2019)',
      'rank': '358.', 
      'runtime': '97 min', 
      'rating': '7.0', 
      'votes': '34,093'
      'genres': 'Animation, Adventure, Comedy'
}

In order to implement this function read_m_by_voting(first_year, last_year, top_number), you need to first implement the function read_m_from_url(url, num_of_m=50). This read_m_from_url function is used to extract num_of_m number of movies from a URL. It will also return a list of dictionaries, each of which represents a movie. As described above, to extract say top 120 movies, you need to parse 3 webpage because each page includes only 50 movies. This read_m_from_url function allows you to extract a specific number of movies from a URL. 

After you implement “read_m_by_voting”, which will return a list of top movies, you need to implement the function write_movies_csv(final_list, filename) to write the movies list to a csv file.

***
You probably want to first work on read_m_from_url, then read_m_by_voting, and then write_movies_csv. For each of the functions, I included a test function. For instance, for the function, read_m_from_url, I have included a test function called test_read_m_from_url(). Please un-comment the test function to test your code. The test function test_write_movies_csv () outputs "IMDb_TopVoted.csv". You need to test each function before moving onto the next. Even within a function, you may need to use print() to test your code very carefully.

***

After you done with scraping the needed data, you should clean and transform it as needed to make it ready for enriching the given "Movies.csv" dataset.

Finaly, export the enriched dataset to a CSV file:
Use the following naming convention: Project_3_PartA_Group#.csv



### Helper functions
***

The helper functions section includes some functions you need to use when implementing the three functions describe above. This inculdes the implementation of two functions and test function for each helper function: read_html(url), test_read_html(), process_str_with_comma(string), and test_process_str_with_comma(). For instance, running  test_process_str_with_comma() will help you understand what the function process_str_with_comma(string) does.



In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

# Helper functions:

In [2]:
"""
The read_html(url) return the contents of an URL hmtl file as a string.
"""
def read_html(url):
    response = requests.get(url)
    content = response.content
    return content

#===========================================================================

"""
If a string contains comma, use "" to enclose the string to write it to a comma-separated values (CSV) 
file with no issue with the comma.
"""
def process_str_with_comma(string):
    if ',' in string:
        new_string = '"' + string.strip() + '"'
    else:
        new_string = string
    return new_string


### Test  read_html 

In [3]:
def test_read_html():
    print (read_html('https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020'))


In [4]:
test_read_html()

b'\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>Feature Film,\nReleased between 2018-01-01 and 2020-12-31\n(Sorted by Number of Votes Descending) - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if (typeof uex == \'function\') {\n      

### Test  process_str_with_comma 

In [5]:
def test_process_str_with_comma():
    """output:
    string: it is a string
    string: "it is a string, right"
    """
    string = 'it is a string'
    print ("string: " + process_str_with_comma(string))
    string = 'it is a string, right'
    print ("string: " + process_str_with_comma(string))


In [6]:
test_process_str_with_comma()

string: it is a string
string: "it is a string, right"


***

## read_m_from_url

Inside this function, you need to write your code to pull the movies information.
For each movie, you need to pull :
- movie_id
- title 
- rank
- year
- runtime
- rating
- votes 
- genres

To give examples on how to pull data from web bage, I have included the code to pull the movie_id.
You need to inculde your code to pull the other needed movie information (title, rank, year, ......). You should have no missing values for each of the collected data.

In [7]:
def read_m_from_url(url, num_of_m=50):
    print(url)
    # this function, read a number of movies from a url. The default value is 50
    
    html_string = read_html(url) # given a url you need to read the hmtl file as a string. 
    # I have included  the read_html function in the helper functions. Please take a look.
    
    # create a soup object
    soup = BeautifulSoup(html_string, "html.parser")
    
    '''
    Click the URL and investigate how you can pull movie_id, rank, title,... from the webpage.
    
    To investigate the html of a web page , For example:
    URL: https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
    Right-click anywhere on the webpage, and at the very bottom of the menu that pops up, 
    you will see "Inspect", Click on it.
    '''
    
# Fetching a div that includes all the movies. This can be done by using find and find_all functions.
    # for example, find_all('div') will give you all divs on the page. Actually, 
    # this find or find_all function can have two parameters,
    # in the code below 'div' is the tag name and 'lister-list' is an attribute value of the tag. You can also do
    # movie_list = soup.find('div', 'lister-list'). Here you explicitly say: I want to find a div with 
    # attribute class = 'lister-list'.
    
    # Since on each imdb page, there's only one div with class = 'lister-list', we can use find rather than find_all. 
    # Find_all will return a list of div tags, while find() will return only one div.
    
    movie_list = soup.find('div', 'lister-list') # this div contains all the listed movies in 
                                                 # the requested html web page.
    
    list_movies = [] # initialize the function return value, which is a list of movies. 
                     # This list will contains the scraped data transformed to a structured format.
    
    # Using count track the number of movies processed. now it's 0 - No movie has been processed yet.
    count = 0
    
    # each movie listed in a div with attribute value 'lister-item mode-advanced'.
    divs=  movie_list.find_all('div','lister-item mode-advanced') # To find all the listed movies in the page.
    for d in divs:
        dict_each_movie = {}  # initialize the movie dictionary to store the movie information.

        # Pulling the movie_id
        try:
            h = d.find('h3','lister-item-header') 
            movie_id= h.find('a').attrs['href']
            movie_id= movie_id[7:-1]
            
        except:
            movie_id=""
        finally:
            dict_each_movie["movie_id"] = movie_id
            print(movie_id)
            
            

        # Pulling the title
        '''  Your code here ...    '''
        try:
            h = d.find('h3','lister-item-header') 
            title= h.find('a').text
            
        except:
            title=""
        finally:
            dict_each_movie["title"] = title
            print(title)
            
           
        
        # Pulling the rank
        '''  Your code here ...    '''
        try:
            rank = d.find('span','lister-item-index unbold text-primary').text.split('.')
            rank = rank[0]
        except:
            rank=""
        finally:
            dict_each_movie["rank"] = rank
            print (rank)           

        
        
        # Pulling the year 
        '''  Your code here ...    '''
        try:
            year = d.find('span','lister-item-year text-muted unbold').text.split(" ")
            year = year[-1]
        except:
            year=""
        finally:
            dict_each_movie["year"] = year
            print (year)           



        # Pulling the runtime
        '''  Your code here ...    '''
        try:
            runtime = d.find('span','runtime').text
        except:
            runtime=""
        finally:
            dict_each_movie["runtime"] = runtime
            print (runtime)     

        
        # Pulling the rating
        '''  Your code here ...    '''
        try:
            rating=  d.find("div", {"name": "ir"}).get('data-value')
        except:
            rating = ""
        finally:
            dict_each_movie["rating"] = rating
            print(rating) 
                     

        # Pulling the votes
        '''  Your code here ...    '''
        try:
            votes =  d.find("p", "sort-num_votes-visible").find_next("span",'text-muted').find_next("span").text.strip()
        except:
            votes = ""
        finally:
            dict_each_movie["votes"] = votes
            print(votes) 

        
        # Pulling the genres
        '''  Your code here ...    '''
        try:
            genres=  d.find('span','lister-item-year text-muted unbold').find_next("span", "genre").text.replace('\n', '').strip()
        except:
            genres = ""
        finally:
            dict_each_movie["genres"] = genres
            print(genres) 

            
        list_movies.append(dict_each_movie)  # To add the movie information to the movies list.

        count +=1
        print('===============================')
        print()
        if count == num_of_m:
            break # to exit from the loop.

    return list_movies


### Test  read_m_from_url

In [8]:

def test_read_m_from_url():
    """ output:
    Movies list: [{'movie_id': 'tt7286456', 'title': 'Joker', 'year': '(2019)', 'rank': '1.', 'runtime': '122 min', 'rating': '8.4', 'votes': '1,074,230'}, {'movie_id': 'tt4154796', 'title': 'Avengers: Endgame', 'year': '(2019)', 'rank': '2.', 'runtime': '181 min', 'rating': '8.4', 'votes': '945,461'}, {'movie_id': 'tt4154756', 'title': 'Avengers: Infinity War', 'year': '(2018)', 'rank': '3.', 'runtime': '149 min', 'rating': '8.4', 'votes': '928,596'}, {'movie_id': 'tt1825683', 'title': 'Black Panther', 'year': '(2018)', 'rank': '4.', 'runtime': '134 min', 'rating': '7.3', 'votes': '678,964'}, {'movie_id': 'tt6751668', 'title': 'Parasite', 'year': '(2019)', 'rank': '5.', 'runtime': '132 min', 'rating': '8.6', 'votes': '666,646'}, {'movie_id': 'tt7131622', 'title': 'Once Upon a Time... In Hollywood', 'year': '(2019)', 'rank': '6.', 'runtime': '161 min', 'rating': '7.6', 'votes': '642,048'}, {'movie_id': 'tt8946378', 'title': 'Knives Out', 'year': '(2019)', 'rank': '7.', 'runtime': '130 min', 'rating': '7.9', 'votes': '534,299'}, {'movie_id': 'tt5463162', 'title': 'Deadpool 2', 'year': '(2018)', 'rank': '8.', 'runtime': '119 min', 'rating': '7.7', 'votes': '519,760'}, {'movie_id': 'tt8579674', 'title': '1917', 'year': '(2019)', 'rank': '9.', 'runtime': '119 min', 'rating': '8.3', 'votes': '495,380'}, {'movie_id': 'tt4154664', 'title': 'Captain Marvel', 'year': '(2019)', 'rank': '10.', 'runtime': '123 min', 'rating': '6.8', 'votes': '493,817'}, {'movie_id': 'tt1727824', 'title': 'Bohemian Rhapsody', 'year': '(2018)', 'rank': '11.', 'runtime': '134 min', 'rating': '7.9', 'votes': '489,064'}, {'movie_id': 'tt6644200', 'title': 'A Quiet Place', 'year': '(2018)', 'rank': '12.', 'runtime': '90 min', 'rating': '7.5', 'votes': '482,141'}, {'movie_id': 'tt4633694', 'title': 'Spider-Man: Into the Spider-Verse', 'year': '(2018)', 'rank': '13.', 'runtime': '117 min', 'rating': '8.4', 'votes': '430,153'}, {'movie_id': 'tt6966692', 'title': 'Green Book', 'year': '(2018)', 'rank': '14.', 'runtime': '130 min', 'rating': '8.2', 'votes': '428,762'}, {'movie_id': 'tt6723592', 'title': 'Tenet', 'year': '(2020)', 'rank': '15.', 'runtime': '150 min', 'rating': '7.4', 'votes': '426,125'}, {'movie_id': 'tt1477834', 'title': 'Aquaman', 'year': '(2018)', 'rank': '16.', 'runtime': '143 min', 'rating': '6.9', 'votes': '417,286'}, {'movie_id': 'tt1270797', 'title': 'Venom', 'year': '(2018)', 'rank': '17.', 'runtime': '112 min', 'rating': '6.7', 'votes': '410,565'}, {'movie_id': 'tt2527338', 'title': 'Star Wars: The Rise Of Skywalker', 'year': '(2019)', 'rank': '18.', 'runtime': '141 min', 'rating': '6.5', 'votes': '404,527'}, {'movie_id': 'tt1677720', 'title': 'Ready Player One', 'year': '(2018)', 'rank': '19.', 'runtime': '140 min', 'rating': '7.4', 'votes': '398,599'}, {'movie_id': 'tt6320628', 'title': 'Spider-Man: Far from Home', 'year': '(2019)', 'rank': '20.', 'runtime': '129 min', 'rating': '7.4', 'votes': '383,087'}, {'movie_id': 'tt1517451', 'title': 'A Star Is Born', 'year': '(2018)', 'rank': '21.', 'runtime': '136 min', 'rating': '7.6', 'votes': '356,745'}, {'movie_id': 'tt1302006', 'title': 'The Irishman', 'year': '(2019)', 'rank': '22.', 'runtime': '209 min', 'rating': '7.8', 'votes': '352,691'}, {'movie_id': 'tt5095030', 'title': 'Ant-Man and the Wasp', 'year': '(2018)', 'rank': '23.', 'runtime': '118 min', 'rating': '7.0', 'votes': '346,822'}, {'movie_id': 'tt2584384', 'title': 'Jojo Rabbit', 'year': '(2019)', 'rank': '24.', 'runtime': '108 min', 'rating': '7.9', 'votes': '340,733'}, {'movie_id': 'tt1950186', 'title': 'Ford v Ferrari', 'year': '(2019)', 'rank': '25.', 'runtime': '152 min', 'rating': '8.1', 'votes': '337,490'}, {'movie_id': 'tt3778644', 'title': 'Solo: A Star Wars Story', 'year': '(2018)', 'rank': '26.', 'runtime': '135 min', 'rating': '6.9', 'votes': '313,683'}, {'movie_id': 'tt4912910', 'title': 'Mission: Impossible - Fallout', 'year': '(2018)', 'rank': '27.', 'runtime': '147 min', 'rating': '7.7', 'votes': '308,761'}, {'movie_id': 'tt6146586', 'title': 'John Wick: Chapter 3 - Parabellum', 'year': '(2019)', 'rank': '28.', 'runtime': '130 min', 'rating': '7.4', 'votes': '306,871'}, {'movie_id': 'tt2737304', 'title': 'Bird Box', 'year': '(2018)', 'rank': '29.', 'runtime': '124 min', 'rating': '6.6', 'votes': '305,135'}, {'movie_id': 'tt2798920', 'title': 'Annihilation', 'year': '(I) (2018)', 'rank': '30.', 'runtime': '115 min', 'rating': '6.8', 'votes': '298,808'}, {'movie_id': 'tt0448115', 'title': 'Shazam!', 'year': '(2019)', 'rank': '31.', 'runtime': '132 min', 'rating': '7.0', 'votes': '295,848'}, {'movie_id': 'tt4881806', 'title': 'Jurassic World: Fallen Kingdom', 'year': '(2018)', 'rank': '32.', 'runtime': '128 min', 'rating': '6.2', 'votes': '283,862'}, {'movie_id': 'tt8367814', 'title': 'The Gentlemen', 'year': '(2019)', 'rank': '33.', 'runtime': '113 min', 'rating': '7.8', 'votes': '282,900'}, {'movie_id': 'tt2948372', 'title': 'Soul', 'year': '(2020)', 'rank': '34.', 'runtime': '100 min', 'rating': '8.1', 'votes': '280,267'}, {'movie_id': 'tt7653254', 'title': 'Marriage Story', 'year': '(2019)', 'rank': '35.', 'runtime': '137 min', 'rating': '7.9', 'votes': '274,033'}, {'movie_id': 'tt7784604', 'title': 'Hereditary', 'year': '(2018)', 'rank': '36.', 'runtime': '127 min', 'rating': '7.3', 'votes': '270,157'}, {'movie_id': 'tt3606756', 'title': 'Incredibles 2', 'year': '(2018)', 'rank': '37.', 'runtime': '118 min', 'rating': '7.6', 'votes': '269,810'}, {'movie_id': 'tt6857112', 'title': 'Us', 'year': '(II) (2019)', 'rank': '38.', 'runtime': '116 min', 'rating': '6.8', 'votes': '257,673'}, {'movie_id': 'tt8772262', 'title': 'Midsommar', 'year': '(2019)', 'rank': '39.', 'runtime': '148 min', 'rating': '7.1', 'votes': '255,303'}, {'movie_id': 'tt0437086', 'title': 'Alita: Battle Angel', 'year': '(2019)', 'rank': '40.', 'runtime': '122 min', 'rating': '7.3', 'votes': '247,522'}, {'movie_id': 'tt5727208', 'title': 'Uncut Gems', 'year': '(2019)', 'rank': '41.', 'runtime': '135 min', 'rating': '7.4', 'votes': '246,741'}, {'movie_id': 'tt6139732', 'title': 'Aladdin', 'year': '(2019)', 'rank': '42.', 'runtime': '128 min', 'rating': '6.9', 'votes': '245,331'}, {'movie_id': 'tt7349662', 'title': 'BlacKkKlansman', 'year': '(2018)', 'rank': '43.', 'runtime': '135 min', 'rating': '7.5', 'votes': '242,325'}, {'movie_id': 'tt4123430', 'title': 'Fantastic Beasts: The Crimes of Grindelwald', 'year': '(2018)', 'rank': '44.', 'runtime': '134 min', 'rating': '6.5', 'votes': '239,448'}, {'movie_id': 'tt7126948', 'title': 'Wonder Woman 1984', 'year': '(2020)', 'rank': '45.', 'runtime': '151 min', 'rating': '5.4', 'votes': '232,962'}, {'movie_id': 'tt7349950', 'title': 'It Chapter Two', 'year': '(2019)', 'rank': '46.', 'runtime': '169 min', 'rating': '6.5', 'votes': '230,874'}, {'movie_id': 'tt6105098', 'title': 'The Lion King', 'year': '(2019)', 'rank': '47.', 'runtime': '118 min', 'rating': '6.8', 'votes': '226,997'}, {'movie_id': 'tt6823368', 'title': 'Glass', 'year': '(2019)', 'rank': '48.', 'runtime': '129 min', 'rating': '6.6', 'votes': '224,677'}, {'movie_id': 'tt1979376', 'title': 'Toy Story 4', 'year': '(2019)', 'rank': '49.', 'runtime': '100 min', 'rating': '7.7', 'votes': '223,650'}, {'movie_id': 'tt2704998', 'title': 'Game Night', 'year': '(I) (2018)', 'rank': '50.', 'runtime': '100 min', 'rating': '6.9', 'votes': '220,159'}]    
    """
    url = "http://www.imdb.com/search/title?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020"
    print ("Movies list: ", read_m_from_url(url))

In [9]:
 test_read_m_from_url()

http://www.imdb.com/search/title?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1
(2019)
122 min
8.4
1,184,668
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2
(2019)
181 min
8.4
1,044,154
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3
(2018)
149 min
8.4
1,007,660
Action, Adventure, Sci-Fi

tt6751668
Parasite
4
(2019)
132 min
8.5
738,984
Comedy, Drama, Horror

tt1825683
Black Panther
5
(2018)
134 min
7.3
718,371
Action, Adventure, Sci-Fi

tt7131622
Once Upon a Time... In Hollywood
6
(2019)
161 min
7.6
685,305
Comedy, Drama

tt8946378
Knives Out
7
(2019)
130 min
7.9
587,277
Comedy, Crime, Drama

tt5463162
Deadpool 2
8
(2018)
119 min
7.7
551,426
Action, Adventure, Comedy

tt8579674
1917
9
(2019)
119 min
8.2
546,926
Action, Drama, War

tt4154664
Captain Marvel
10
(2019)
123 min
6.8
530,344
Action, Adventure, Sci-Fi

tt1727824
Bohemian Rhapsody
11
(2018)
134 min
7.9
515,252
Biography, Drama, Music

tt6644200
A Quiet Place
12
(2018)
90 

***
##  read_m_by_voting

This method takes two years (first_year, last_year) and a number (top_number) as inputs. 
If we have first year = 2018, and last_year =2020, and top_number = 500, we want to retrieve top 500 movies 
that were released between 2018 and 2020, we need to do two things:

##### 1. Construct a url. An example url can be 
    https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
    
        This URL means that the web page will display movies based on user_voting in descending 
        order "sort=num_votes,desc" between year 2018 and 2020,
        It will start from the first movie (i.e., start=1 - we need to set the start index in the url).
    
        IMDB will display just 50 movies based on this URL. So in order to review more 
        movies, we need to click "next" on the web page. By clicking the next button,
        we will see a new url
           https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=51&title_type=feature&year=2018,2020

        If we compare the two urls above, we can easily see that in the second url, start=51, i.e., IMDB provides 
        another 50 movies, starting from movie No 51.If we want to retrieve the top 61 movies, we need to open 
        two web pages with two urls. 
        And if we want to retrieve top 256 movies, we need to open 6 different URLs.
        Obviously, we want to use a loop to construct the different URLs.
    
    
##### 2. Read movies from the URL: using "read_m_from_url" method. 
      What this method does is that it opens a url and read numbers_of_movies_you_need_to_read_on_the_page. 
      For example, if we want to read top 61 movies, we will need to first open 
      a url https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
      
      We need to read all 50 movies on the page by calling read_m_from_url(url), and then we need to open 
      the second url http://www.imdb.com/search/title?at=0&sort=user_rating&start=1&title_type=feature&year=2005,2016, but 
      we just need to read 61-50=11 movies from the page. Now, let's set current_index = 51 and top_number = 61, we 
      actually need to retrieve (top_number - current_index + 1) movies. we can do this by calling
      read_m_from_url(url, top_number - the current_index+1).
      
      Hence, we need to use if statement here. Based on top_number, we may need to open multiple urls 
         (e.g., if top 256, we need to open 6 urls)
    

In [10]:
m_per_page = 50 # by default, imdb return 50 movies page url.
def read_m_by_voting(first_year, last_year, top_number):
    
    current_index = 1  # initialize current_index. In the first iteration, we need to have start = 1.
    
    final_list = []  # initialize the return value. This method returns a list. Each item in the list is a dictionary. 
                     # Each dictionary includes information regarding a movie.

    for i in range(int(math.ceil(top_number/50.0))):
        url= 'http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start='+str(current_index)+'&title_type=feature&year='+str(first_year)+','+ str(last_year)

        if (i+1) == len(range( int(math.ceil(top_number/50.0)))):
            lis = read_m_from_url(url, top_number - current_index + 1)
        else:
            lis = read_m_from_url(url, m_per_page)
        final_list += lis
        current_index +=50

    return final_list

### Test read_m_by_voting

In [11]:
def test_read_m_by_voting():
    """output:
    [{'movie_id': 'tt7286456', 'title': 'Joker', 'year': '(2019)', 'rank': '1.', 'genres': 'Crime, Drama, Thriller', 'runtime': '122 min', 'rating': '8.4', 'votes': '"1,074,179"'}, {'movie_id': 'tt4154796', 'title': 'Avengers: Endgame', 'year': '(2019)', 'rank': '2.', 'genres': 'Action, Adventure, Drama', 'runtime': '181 min', 'rating': '8.4', 'votes': '"945,422"'}]
    """
    print (read_m_by_voting(2018,2020,3))  # This will print a list of top three movies.

In [12]:
test_read_m_by_voting()

http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1
(2019)
122 min
8.4
1,184,668
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2
(2019)
181 min
8.4
1,044,154
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3
(2018)
149 min
8.4
1,007,660
Action, Adventure, Sci-Fi

[{'movie_id': 'tt7286456', 'title': 'Joker', 'rank': '1', 'year': '(2019)', 'runtime': '122 min', 'rating': '8.4', 'votes': '1,184,668', 'genres': 'Crime, Drama, Thriller'}, {'movie_id': 'tt4154796', 'title': 'Avengers: Endgame', 'rank': '2', 'year': '(2019)', 'runtime': '181 min', 'rating': '8.4', 'votes': '1,044,154', 'genres': 'Action, Adventure, Drama'}, {'movie_id': 'tt4154756', 'title': 'Avengers: Infinity War', 'rank': '3', 'year': '(2018)', 'runtime': '149 min', 'rating': '8.4', 'votes': '1,007,660', 'genres': 'Action, Adventure, Sci-Fi'}]


***
# write_movies_csv
This method to write the movies to a csv file.
Each row in the csv file represents a movie. 
 - The parameter final_list includes a number of movies, which is the output of the function read_m_by_voting. 
 - The filename represents the output file name.
 
 ***
 Important note:  make sure to complete the code of read_m_from_url method before running write_movies_csv method.
 ***

In [13]:

def write_movies_csv(final_list, filename):

    lis = [] # to write the file, we create a list of strings.
    
    header = "movie_id,"+"rank,"+"title,"+"year,"+"rating,"+"runtime,"+"votes,"+"genres"
    lis.append(header) # add the header to the list
    for movie in final_list:
        string =  process_str_with_comma(movie["movie_id"]) +"," + process_str_with_comma(movie["rank"]) +","+ process_str_with_comma(movie["title"]) +","+ process_str_with_comma(movie["year"]) + ","+  process_str_with_comma(movie["rating"]) +","+ process_str_with_comma(movie["runtime"]) + "," + process_str_with_comma(movie["votes"]) + "," + process_str_with_comma(movie["genres"])
        print (string)
        lis.append(string)# add the string to the list

    # Writing the strings list to csv file
    f = None
    f = open(filename,"w")
    for s in lis:
        f.write("%s\n" % s)
    f.close()
    
    return


In [14]:
# The output of the test_write_movies_csv method is the "IMDb_TopVoted.csv" file.
def test_write_movies_csv(): 
    li = read_m_by_voting(2018, 2020, 500) # To read the top voted 500 movies between 2018 and 2020 from imdb.
    print(li)
    print("================================================================")
    write_movies_csv(li,"IMDb_TopVoted_Group15.csv") 


In [15]:
test_write_movies_csv()

http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1
(2019)
122 min
8.4
1,184,668
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2
(2019)
181 min
8.4
1,044,154
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3
(2018)
149 min
8.4
1,007,660
Action, Adventure, Sci-Fi

tt6751668
Parasite
4
(2019)
132 min
8.5
738,984
Comedy, Drama, Horror

tt1825683
Black Panther
5
(2018)
134 min
7.3
718,371
Action, Adventure, Sci-Fi

tt7131622
Once Upon a Time... In Hollywood
6
(2019)
161 min
7.6
685,305
Comedy, Drama

tt8946378
Knives Out
7
(2019)
130 min
7.9
587,277
Comedy, Crime, Drama

tt5463162
Deadpool 2
8
(2018)
119 min
7.7
551,426
Action, Adventure, Comedy

tt8579674
1917
9
(2019)
119 min
8.2
546,926
Action, Drama, War

tt4154664
Captain Marvel
10
(2019)
123 min
6.8
530,344
Action, Adventure, Sci-Fi

tt1727824
Bohemian Rhapsody
11
(2018)
134 min
7.9
515,252
Biography, Drama, Music

tt6644200
A Quiet Place
12
(2018)
90

tt6266538
Vice
101
(2018)
132 min
7.2
141,530
Biography, Comedy, Drama

tt4729430
Klaus
102
(2019)
96 min
8.1
141,341
Animation, Adventure, Comedy

tt7146812
Onward
103
(2020)
102 min
7.4
140,740
Animation, Adventure, Comedy

tt7798634
Ready or Not
104
(2019)
95 min
6.8
140,228
Action, Comedy, Horror

tt13143964
Borat Subsequent Moviefilm
105
(2020)
95 min
6.6
136,428
Comedy

tt5814060
The Nun
106
(2018)
96 min
5.3
136,190
Horror, Mystery, Thriller

tt10272386
The Father
107
(2020)
97 min
8.3
133,722
Drama, Mystery

tt4500922
Maze Runner: The Death Cure
108
(2018)
143 min
6.2
133,160
Action, Adventure, Sci-Fi

tt7959026
The Mule
109
(2018)
116 min
7
132,665
Crime, Drama, Thriller

tt2283336
Men in Black: International
110
(2019)
114 min
5.6
130,101
Action, Adventure, Comedy

tt2854926
Tag
111
(2018)
100 min
6.5
129,993
Action, Comedy

tt8110330
Dil Bechara
112
(2020)
101 min
8.1
128,453
Comedy, Drama, Romance

tt9893250
I Care a Lot
113
(2020)
118 min
6.3
127,707
Comedy, Crime, Drama



tt4003440
The House That Jack Built
201
(2018)
152 min
6.8
72,310
Crime, Drama, Horror

tt8350360
Annabelle Comes Home
202
(2019)
106 min
5.9
72,302
Horror, Mystery, Thriller

tt4532826
Robin Hood
203
(2018)
116 min
5.3
72,257
Action, Adventure, Drama

tt6921996
Johnny English Strikes Again
204
(2018)
89 min
6.2
72,256
Action, Adventure, Comedy

tt1226837
Beautiful Boy
205
(2018)
120 min
7.3
71,540
Biography, Drama

tt2990140
The Christmas Chronicles
206
(2018)
104 min
7
71,356
Adventure, Comedy, Family

tt10618286
Mank
207
(2020)
131 min
6.8
71,328
Biography, Comedy, Drama

tt1137450
Death Wish
208
(2018)
107 min
6.3
71,310
Action, Crime, Drama

tt7838252
K.G.F: Chapter 1
209
(2018)
156 min
8.2
71,175
Action, Crime, Drama

tt8155288
Happy Death Day 2U
210
(2019)
100 min
6.2
71,099
Comedy, Horror, Mystery

tt5220122
Hotel Transylvania 3: Summer Vacation
211
(2018)
97 min
6.3
70,990
Animation, Adventure, Comedy

tt4332232
Fractured
212
(2019)
99 min
6.4
70,761
Thriller

tt5033998
Charli

tt6079516
I See You
301
(2019)
98 min
6.8
47,039
Crime, Drama, Horror

tt5177088
The Girl in the Spider's Web
302
(2018)
115 min
6.1
47,037
Action, Crime, Drama

tt5116302
Togo
303
(2019)
113 min
7.9
46,879
Adventure, Biography, Drama

tt10431500
Miracle in Cell No. 7
304
(2019)
132 min
8.2
46,833
Drama

tt5814534
Spies in Disguise
305
(2019)
102 min
6.8
46,530
Animation, Action, Adventure

tt7734218
Stuber
306
(2019)
93 min
6.1
46,474
Action, Comedy, Crime

tt7431594
Race 3
307
(2018)
160 min
1.9
45,989
Action, Crime, Thriller

tt5431890
Official Secrets
308
(2019)
112 min
7.3
45,874
Biography, Crime, Drama

tt7153766
Unsane
309
(2018)
98 min
6.4
45,527
Drama, Horror, Mystery

tt5073642
Color Out of Space
310
(2019)
111 min
6.1
45,189
Horror, Mystery, Sci-Fi

tt2837574
The Old Man & the Gun
311
(2018)
93 min
6.7
45,083
Biography, Comedy, Crime

tt6491178
Dragged Across Concrete
312
(2018)
159 min
6.9
44,892
Action, Crime, Drama

tt11655202
Riders of Justice
313
(2020)
116 min
7.5
44,8

tt10324144
Article 15
401
(2019)
130 min
8.2
32,390
Crime, Drama, Mystery

tt4761916
Unfriended: Dark Web
402
(2018)
92 min
6
32,105
Horror, Mystery, Thriller

tt8108268
The Tashkent Files
403
(2019)
134 min
8.1
32,042
Drama, Mystery, Thriller

tt9806192
I Lost My Body
404
(2019)
81 min
7.5
31,854
Animation, Drama, Fantasy

tt3833480
The Outpost
405
(2019)
123 min
6.8
31,765
Action, Drama, History

tt4073790
The Darkest Minds
406
(2018)
104 min
5.7
31,702
Action, Adventure, Drama

tt8633462
Quo Vadis, Aida?
407
(2020)
101 min
8
31,541
Drama, History, War

tt9398640
Between Two Ferns: The Movie
408
(2019)
82 min
6.1
31,538
Comedy

tt8983202
Kabir Singh
409
(2019)
173 min
7
31,363
Action, Drama, Romance

tt6513656
Escape Plan 2: Hades
410
(2018)
96 min
3.8
31,351
Action, Crime, Mystery

tt5935704
Padmaavat
411
(2018)
164 min
7
31,311
Drama, History, Romance

tt10612922
One Night in Miami...
412
(2020)
114 min
7.1
31,225
Drama

tt8850222
Peninsula
413
(2020)
116 min
5.5
31,172
Action, Hor

# Importing the given dataset "Movies.csv" to Pandas DataFrame called df1

In [16]:
# Importing the csv file to df1 and print the df1.

'''  Your code here ...    '''
df1 = pd.read_csv("Movies.csv")
print(df1)

       movie_id titleType           originalTitle  isAdult
0     tt7286456     movie                   Joker        0
1     tt4154796     movie       Avengers: Endgame        0
2     tt4154756     movie  Avengers: Infinity War        0
3     tt6751668     movie            Gisaengchung        0
4     tt1825683     movie           Black Panther        0
..          ...       ...                     ...      ...
495   tt8266310     movie    Blinded by the Light        0
496   tt5316540     movie                   Close        0
497   tt9340860     movie              Let Him Go        0
498  tt10199590     movie          Les misérables        0
499   tt6628102     movie             Ahlat Agaci        0

[500 rows x 4 columns]


# Import the scraped data from the IMDb_TopVoted.csv file to Pandas DataFrame called df2

In [17]:
# You need to import the collected dataset "IMDb_TopVoted.csv".
# To handel Latin characters that may contained in the csv file
# with no issue, use  encoding= "ISO-8859-1" with the pd.read_csv()
# Example: df1 = pd.read_csv('thefilename.csv', encoding= "ISO-8859-1") 
# Using encoding= "ISO-8859-1" will avoid Unicode-Decode-Errors.

'''  Your code here ...    '''
df2 = pd.read_csv('IMDb_TopVoted_Group15.csv', encoding="ISO-8859-1")
print(df2)

       movie_id  rank                   title    year  rating  runtime  \
0     tt7286456     1                   Joker  (2019)     8.4  122 min   
1     tt4154796     2       Avengers: Endgame  (2019)     8.4  181 min   
2     tt4154756     3  Avengers: Infinity War  (2018)     8.4  149 min   
3     tt6751668     4                Parasite  (2019)     8.5  132 min   
4     tt1825683     5           Black Panther  (2018)     7.3  134 min   
..          ...   ...                     ...     ...     ...      ...   
495   tt8266310   496    Blinded by the Light  (2019)     6.9  118 min   
496   tt5316540   497                   Close  (2019)     5.7   94 min   
497   tt9340860   498              Let Him Go  (2020)     6.7  113 min   
498  tt10199590   499         Les MisÃ©rables  (2019)     7.6  104 min   
499   tt8652728   500                   Waves  (2019)     7.5  135 min   

         votes                     genres  
0    1,184,668     Crime, Drama, Thriller  
1    1,044,154   Action

## Data cleansing and transformation for df2.

In [18]:
# Cleaning and tranforming df2
 # rank, year, runtime, and votes should have a numeric integer data type.

'''  Your code here ...    '''
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  500 non-null    object 
 1   rank      500 non-null    int64  
 2   title     500 non-null    object 
 3   year      500 non-null    object 
 4   rating    500 non-null    float64
 5   runtime   500 non-null    object 
 6   votes     500 non-null    object 
 7   genres    500 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 31.4+ KB


In [19]:
# Printing unique values for list of columns
for col in ["rank", "year","runtime", "votes"]:
    print(col)

    # get a list of unique values
    unique = df2[col].unique()
    print('#.of unique values: ', len(unique), '\nList of unique values: ',unique, '\n*************************************\n')

rank
#.of unique values:  500 
List of unique values:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 

In [20]:
#Check duplicated row
df2.duplicated().sum()

0

In [21]:
# Number of missing values in each coulmn
df2.isnull().sum()

movie_id    0
rank        0
title       0
year        0
rating      0
runtime     0
votes       0
genres      0
dtype: int64

### For rank
- we have gotten integer type of rank when scrapping rank from web. 
- The code we use above is "rank = d.find('span','lister-item-index unbold text-primary').text.split('.')"

### For year
- we need to remove brackets in 'year' variable and change it into integer, eg, changing (2019) into 2019

In [22]:
df2['year'] = df2['year'].str.strip('()').astype(int)

### For runtime
- we need to remove string 'min' in 'runtime' variable and change it into integer. 

In [23]:
df2['runtime'] = df2['runtime'].str.strip(' min').astype(int)

### For votes
- we need to remove comma in 'votes' variable and change it into integer

In [24]:
df2['votes'] = df2['votes'].str.replace(',',"").astype(int)

### Check our data type again

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  500 non-null    object 
 1   rank      500 non-null    int64  
 2   title     500 non-null    object 
 3   year      500 non-null    int64  
 4   rating    500 non-null    float64
 5   runtime   500 non-null    int64  
 6   votes     500 non-null    int64  
 7   genres    500 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 31.4+ KB


# 	Enrich the given dataset (df1) by merging it to the scraped data (df2).

In [26]:
# Merege the two dataframes to one dataframe called df.
'''  Your code here ...    '''
df = pd.merge(left=df1, right=df2,on='movie_id')
df

,movie_id,titleType,originalTitle,isAdult,rank,title,year,rating,runtime,votes,genres
0,tt7286456,movie,Joker,0,1,Joker,2019,8.4,122,1184668,"Crime, Drama, Thriller"
1,tt4154796,movie,Avengers: Endgame,0,2,Avengers: Endgame,2019,8.4,181,1044154,"Action, Adventure, Drama"
2,tt4154756,movie,Avengers: Infinity War,0,3,Avengers: Infinity War,2018,8.4,149,1007660,"Action, Adventure, Sci-Fi"
3,tt6751668,movie,Gisaengchung,0,4,Parasite,2019,8.5,132,738984,"Comedy, Drama, Horror"
4,tt1825683,movie,Black Panther,0,5,Black Panther,2018,7.3,134,718371,"Action, Adventure, Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...
494,tt6521876,movie,Jay and Silent Bob Reboot,0,495,Jay and Silent Bob Reboot,2019,5.6,105,24779,"Action, Adventure, Comedy"
495,tt8266310,movie,Blinded by the Light,0,496,Blinded by the Light,2019,6.9,118,24537,"Comedy, Drama, Music"
496,tt5316540,movie,Close,0,497,Close,2019,5.7,94,24400,"Action, Drama, Thriller"
497,tt9340860,movie,Let Him Go,0,498,Let Him Go,2020,6.7,113,24350,"Crime, Drama, Thriller"


### Rearrange the dataset fields to be listed in the following order: 
 movie_id, rank, votes, title, originalTitle, year, rating, titleType, isAdult, runtime,  genres

In [27]:
# Rearrange the dataset fields.
'''  Your code here ...    '''
col_order = ["movie_id", "rank", "votes", "title", "originalTitle", "year", "rating", "titleType", "isAdult", "runtime","genres"]
df = df[col_order]
df

,movie_id,rank,votes,title,originalTitle,year,rating,titleType,isAdult,runtime,genres
0,tt7286456,1,1184668,Joker,Joker,2019,8.4,movie,0,122,"Crime, Drama, Thriller"
1,tt4154796,2,1044154,Avengers: Endgame,Avengers: Endgame,2019,8.4,movie,0,181,"Action, Adventure, Drama"
2,tt4154756,3,1007660,Avengers: Infinity War,Avengers: Infinity War,2018,8.4,movie,0,149,"Action, Adventure, Sci-Fi"
3,tt6751668,4,738984,Parasite,Gisaengchung,2019,8.5,movie,0,132,"Comedy, Drama, Horror"
4,tt1825683,5,718371,Black Panther,Black Panther,2018,7.3,movie,0,134,"Action, Adventure, Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...
494,tt6521876,495,24779,Jay and Silent Bob Reboot,Jay and Silent Bob Reboot,2019,5.6,movie,0,105,"Action, Adventure, Comedy"
495,tt8266310,496,24537,Blinded by the Light,Blinded by the Light,2019,6.9,movie,0,118,"Comedy, Drama, Music"
496,tt5316540,497,24400,Close,Close,2019,5.7,movie,0,94,"Action, Drama, Thriller"
497,tt9340860,498,24350,Let Him Go,Let Him Go,2020,6.7,movie,0,113,"Crime, Drama, Thriller"


# Export the enriched dataset to a CSV file:

In [28]:
# Use the following naming convention: 
#  Project_3_PartA_Group#.csv

'''  Your code here ...    '''
df.to_csv('Project_3_PartA_Group15.csv', index =False)